In [ ]:
import json
from pathlib import Path
from typing import Dict, Any, List, Tuple
import re
from google.colab import files

# Upload the dataset interactively in Colab
uploaded = files.upload()
input_path = list(uploaded.keys())[0]
cleaned_output_path = "nightline_cleaned_final.jsonl"
instruction_output_path = "nightline_instruction_dataset.jsonl"
dpo_output_path = "nightline_dpo_dataset.jsonl"

# Patterns to strip from the start of responses (exclude directive-intended phrases)
STRIP_PATTERNS = [
    r"^(directive|neutral|nightline-aligned)[:\s]+",
    r"^response[:\s]+",
    r"^option \d+[:\s]+",
    r"^\d+\.\s+",
    r"^here is your response[:\s]+",
    r"^as an ai language model[:\s]+",
    r"^i am an ai[:\s]+"
]

def load_jsonl(file_path: str):
    with open(file_path, 'r') as f:
        return [json.loads(line.strip()) for line in f if line.strip()]

def save_jsonl(data: list, output_path: str):
    with open(output_path, 'w') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

def normalize_traits(traits: Dict[str, Any]) -> Dict[str, bool]:
    expected_traits = ['non_directive', 'judgment_free', 'empathic_reflection', 'escalation_safe', 'empathy_score']
    return {key: bool(traits.get(key, False)) if key != 'empathy_score' else int(traits.get(key, 0)) for key in expected_traits}

def clean_text(text: str) -> str:
    lowered = text.lower().strip()
    for pattern in STRIP_PATTERNS:
        if re.match(pattern, lowered):
            return re.sub(pattern, '', text, flags=re.IGNORECASE).strip()
    return text.strip()

def clean_entry(entry: Dict[str, Any]) -> Dict[str, Any]:
    cleaned = {
        "prompt": entry.get("prompt", "").strip(),
        "responses": {}
    }

    for key in ['directive', 'neutral', 'nightline_aligned']:
        response = entry.get("responses", {}).get(key, {})
        text = clean_text(response.get("text", ""))

        cleaned_response = {
            "text": text,
            "traits": normalize_traits(response.get("traits", {}))
        }
        cleaned["responses"][key] = cleaned_response

    return cleaned

def extract_instruction_data(cleaned_data: List[Dict[str, Any]]) -> List[Dict[str, str]]:
    instruction_pairs = []
    for entry in cleaned_data:
        prompt = entry.get("prompt", "").strip()
        aligned = entry.get("responses", {}).get("nightline_aligned", {}).get("text", "").strip()
        if prompt and aligned:
            instruction_pairs.append({"prompt": prompt, "response": aligned})
    return instruction_pairs

def extract_dpo_data(cleaned_data: List[Dict[str, Any]]) -> List[Dict[str, str]]:
    dpo_pairs = []
    for entry in cleaned_data:
        prompt = entry.get("prompt", "").strip()
        aligned = entry.get("responses", {}).get("nightline_aligned", {}).get("text", "").strip()
        directive = entry.get("responses", {}).get("directive", {}).get("text", "").strip()
        neutral = entry.get("responses", {}).get("neutral", {}).get("text", "").strip()

        if prompt and aligned and directive:
            dpo_pairs.append({"prompt": prompt, "chosen": aligned, "rejected": directive})
        if prompt and aligned and neutral:
            dpo_pairs.append({"prompt": prompt, "chosen": aligned, "rejected": neutral})
    return dpo_pairs

def process_dataset(input_path: str, cleaned_output_path: str, instruction_output_path: str, dpo_output_path: str):
    data = load_jsonl(input_path)
    cleaned_data = [clean_entry(entry) for entry in data if entry.get("prompt") and entry.get("responses")]
    save_jsonl(cleaned_data, cleaned_output_path)
    print(f"Cleaned entries: {len(cleaned_data)}")

    instruction_data = extract_instruction_data(cleaned_data)
    save_jsonl(instruction_data, instruction_output_path)
    print(f"Instruction-tuning entries: {len(instruction_data)}")

    dpo_data = extract_dpo_data(cleaned_data)
    save_jsonl(dpo_data, dpo_output_path)
    print(f"DPO pairs generated: {len(dpo_data)}")

    files.download(cleaned_output_path)
    files.download(instruction_output_path)
    files.download(dpo_output_path)

# Run immediately in Colab
process_dataset(input_path, cleaned_output_path, instruction_output_path, dpo_output_path)
